## Harry Durbin
## CE 263-Scalable Spatial Analytics

In [1]:
import numpy as np

from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn import cross_validation

import time
from datetime import datetime, timedelta

import utm
import geopy
from geopy.geocoders import Nominatim

## Predicting Taxi Fares

### splitting data into train and test sets

In [3]:
fn = 'Taxi_Train.csv'
predictions = np.genfromtxt(fn, delimiter=',')

In [4]:
# time_input = []
time_convert = np.zeros(shape=([len(predictions),3]))
day = np.zeros(shape=([len(predictions),1]))
i = 0
with open(fn) as f: 
    for line in f: # go over data file, line by line
        x = line.split(',') # get a list of attributes, as strings
        # parse the time strings from %m/%d/%Y %H:%M format into seconds
        tto = datetime.strptime(x[1], "%m/%d/%Y %H:%M")

        dayofweek = tto.strftime('%a')
        if dayofweek == 'Sat':
            day[i,0] = 1
        elif dayofweek == 'Sun':
            day[i,0] = 1

        to = (tto - tto.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
        time_convert[i,0] = to
        time_convert[i,1] = int(to/60/60)
        time_convert[i,2] = int(to%3600/60)
        i = i+1

In [5]:
# add day column to predictions array
predictions = np.concatenate((predictions,day),axis=1)

In [6]:
# delete row from array if fare value appears to be missing/inaccurate
#i.e. if fare is less than $1.00, it seems unrealistic
predictions = predictions[~(predictions[:,3]<1)] 

In [7]:
# cross validation
kf = cross_validation.KFold(n=len(predictions), n_folds=10, indices=None, 
                       shuffle=False, random_state=None)
for train_index, test_index in kf:
#     print("TRAIN:", train_index, "TEST:", test_index)
    predictions_train, predictions_test = predictions[train_index], predictions[test_index]
    
# cross validation
kf2 = cross_validation.KFold(n=len(time_convert), n_folds=10, indices=None, 
                       shuffle=False, random_state=None)
for train_index, test_index in kf2:
#     print("TRAIN:", train_index, "TEST:", test_index)
    time_convert_train, time_convert_test = time_convert[train_index], time_convert[test_index]  
    

### defining the model

In [8]:
# Travel Time predictor
class TaxiFarePredictor:
     
    def __init__(self, fname, TTmodel_type = 'RF', Fmodel_type = 'RF'):

        self.TTmodel_type = TTmodel_type # travel time model type
        self.Fmodel_type = Fmodel_type # fare predictor model type
        
        self.TTmodel = []
        self.Fmodel = []
        
        # load data and process features
        self.X, self.T, self.F = self.load_data(fname)
    
        # train travel time model
        self.travel_time_model(model_type = self.TTmodel_type)
        
        # train fare model
        self.fare_model(model_type = self.Fmodel_type)
     
    
    def load_data(self, fname):
        
        #  File structure must be 
        #
        #  0-4    ['ID', 'STARTDATE', 'ENDDATE', 'Total_amount', 'PASSENGER_NUM',
        #  5-10   'GPS_START_LO', 'GPS_START_LA', 'GPS_END_LO', 'GPS_END_LA', 'StartTAZ1454', 'TAZ1454_end, day]
        #
        trips = np.genfromtxt(fname, delimiter=',')
        features = np.zeros((trips.shape[0],15))
        
        
        with open(fname,'rU') as f: 

            for line in f: # go over data file, line by line

                l = line.split(',') # get a list of attributes, as strings

                # parse the time strings from %m/%d/%Y %H:%M format into seconds
                tto = datetime.strptime(l[1], "%m/%d/%Y %H:%M")
                ttd = datetime.strptime(l[2], "%m/%d/%Y %H:%M")
                
                dayofweek = tto.strftime('%a')
                if dayofweek == 'Sat':
                    day = 1
                elif dayofweek == 'Sun':
                    day = 1

                # trip duration
                tdelta = time.mktime(ttd.timetuple()) - time.mktime(tto.timetuple())

                # convert to time since midnight
                to = (tto - tto.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
                td = (ttd - ttd.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()

                # convert strings to floats
                l[5:10] = map(lambda x: float(x), l[5:10])

                # convert to meters in UTM
                o = np.array(utm.from_latlon(l[6], l[5])[:2])
                d = np.array(utm.from_latlon(l[8], l[7])[:2])

                # euclidean distance between origin and destination
                dist1 = cdist([o],[d])[0][0]
                dist2 = abs(o[0]-d[0]) + abs(o[1]-d[1])
                
                
                # collect all features we have computed
                # id, fare, start time, travel time,passengers, TAZo, TAZd,end time, distance, origin, dest
                features[int(l[0]),:15] = np.append(np.array([int(l[0]), float(l[3]), td, tdelta, 
                                                              float(l[4]), float(l[9]), float(l[10]),
                                                              to, dist1,dist2,day]), np.append(o, d))

#the following code applies when data is being trained with cross validation
# remove if not bothering to train
########################################################################################       
        # delete row from array if fare data missing/inaccurate
        features = features[~(features[:,1]<1.0)]                       
                
        # cross validation - comment out after parameters determined
        kf = cross_validation.KFold(n=len(features), n_folds=10, indices=None, 
                               shuffle=False, random_state=None)
        for train_index, test_index in kf:
        #     print("TRAIN:", train_index, "TEST:", test_index)
            features_train, features_test = features[train_index], features[test_index]

        # return X, travel time, fare          
        return features_train[:,4:], features_train[:,3], features_train[:,1]   

########################################################################################
#add this code if data is NOT being trained with cross validation
#  #       return X, travel time, fare          
#         return features[:,4:], features[:,3], features[:,1]   

########################################################################################  
    
    def travel_time_model(self, model_type='RF'):
    
        if model_type == 'RF':
            self.TTmodel = RandomForestRegressor(random_state=0, n_estimators=10, max_depth=10)
            self.TTmodel.fit(self.X, self.T)           
        elif model_type == 'DT':
            self.TTmodel = DecisionTreeRegressor(max_depth=100)
            self.TTmodel.fit(self.X, self.T)           
        else:
            print 'Unknown model type for travel times predictor.'            
    
    def travel_time(self, Xq):
    
        return self.TTmodel.predict(Xq)
        
    def fare_model(self, model_type='RF'):
        
        T = self.T
        X = np.hstack( (self.X, np.array([self.T]).T ))
        
        if model_type == 'RF':
            self.Fmodel = RandomForestRegressor(random_state=0, n_estimators=10, max_depth=10)
            self.Fmodel.fit(X, self.F)  
        elif model_type == 'DT':
            self.Fmodel = DecisionTreeRegressor(max_depth=100)
            self.Fmodel.fit(X, self.F)          
        else:
            print 'Unknown model type for trip fare predictor.'
    
    
    def fare(self, Xq):
        
        return self.Fmodel.predict(Xq)

    
    # can take (lat, lon) pairs or address as text, via geolocator geocoding
    # origin, destination, start time, number passenger, TAZ origin, TAZ dest
    def TaxiFare(self, orig, dest, tstart='0:00', npax=1, TAZo=0, TAZd=0, day=0):
        
        if not isinstance(orig, basestring): 
            if not isinstance(dest, basestring): 
                
                o = np.array(utm.from_latlon(orig[1], orig[0])[:2])
                d = np.array(utm.from_latlon(dest[1], dest[0])[:2])
               
        else:
            
            try:
                geolocator = Nominatim()

                origin = geolocator.geocode(orig)
                destination = geolocator.geocode(dest)
    
                o = np.array(utm.from_latlon(origin.latitude, origin.longitude)[:2])
                d = np.array(utm.from_latlon(destination.latitude, destination.longitude)[:2])
            
            except:
                
                print 'Geocoding error. Try providing WGS84 coordinates instead.'
                
                return 0
            
        # euclidean distance between origin and destination
        dist1 = cdist([o],[d])[0][0]
        dist2 = abs(o[0]-d[0]) + abs(o[1]-d[1])

        
        #print str(depart_s)
        temp = datetime.strptime('09/01/2012 '+ str(tstart), "%m/%d/%Y %H:%M")
        depart = (temp - temp.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
       
        # features for travel time prediction
        xq = np.append(np.array([float(npax), float(TAZo), float(TAZd), depart, dist1,dist2,day]), np.append(o, d))
                
        # predict travel time
        tt = self.travel_time(xq)
        
        # predict fare
        f = self.fare(np.append(xq, tt))
        
        return f[0]  # tt[0]

### training the model

In [9]:
TFP = TaxiFarePredictor('Taxi_Train.csv', TTmodel_type = 'RF', Fmodel_type = 'RF')

### predicting fares for kfolded test data

In [10]:
time_input = []
for row in range(len(time_convert_test)):
    time_input.append(str(int(time_convert_test[row,1])) + ':' + str(int(time_convert_test[row,2])))
    

In [11]:
#  0-4    ['ID', 'STARTDATE', 'ENDDATE', 'Total_amount', 'PASSENGER_NUM',
#  5-10   'GPS_START_LO', 'GPS_START_LA', 'GPS_END_LO', 'GPS_END_LA', 'StartTAZ1454', 'TAZ1454_end]
coords_start = predictions_test[:,5:7]
coords_end = predictions_test[:,7:9]
time_start = time_input[:]
passengers = predictions_test[:,4]
taz_start = predictions_test[:,9]
taz_end = predictions_test[:,10]
day = predictions_test[:,11]

In [12]:
predicted_fares = []
cv = []

for row in range(len(predictions_test)):
    predicted_fares.append(TFP.TaxiFare((coords_start[row,0],coords_start[row,1]), (coords_end[row,0],coords_end[row,1]),
                   time_start[row], passengers[row], taz_start[row], taz_end[row],day[row]))
    cv.append(((predicted_fares[row] - predictions_test[row,3])**2)**0.5)
#     if row%200==0:
    

In [13]:
avg_error = np.mean(cv)
print '$',format(avg_error, '.2f')

$ 2.37


In [14]:
actual_fares = predictions_test[:,3]
for row in range(10):
    print '$',format(actual_fares[row],'.2f'), '$',format(predicted_fares[row], '.2f')

$ 42.00 $ 47.22
$ 13.40 $ 12.60
$ 13.95 $ 16.04
$ 11.00 $ 10.62
$ 21.65 $ 12.21
$ 9.30 $ 8.84
$ 21.10 $ 13.42
$ 6.25 $ 6.61
$ 8.45 $ 9.60
$ 6.25 $ 7.70


### running actual test data in the model

In [16]:
test_file = 'Taxi_Query.csv'
time_input_testset = []
day = []
with open(test_file) as fn: 
    for line in fn: # go over data file, line by line
        x = line.split(',') # get a list of attributes, as strings
        # parse the time strings from %m/%d/%Y %H:%M format into seconds
        tto = datetime.strptime(x[1], "%m/%d/%Y %H:%M")
        to = (tto - tto.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
        h = int(to/60/60) 
        m = int(to%3600/60)
        time_input_testset.append(str(h) + ':' + str(m))
        dayofweek = tto.strftime('%a')
        if dayofweek == 'Sat':
            dayx = 1
        elif dayofweek == 'Sun':
            dayx = 1
        else:
            dayx=0
        day.append(dayx)
            

#  0-4    ['ID', 'STARTDATE', 'ENDDATE', 'Total_amount', 'PASSENGER_NUM',
#  5-10   'GPS_START_LO', 'GPS_START_LA', 'GPS_END_LO', 'GPS_END_LA', 'StartTAZ1454', 'TAZ1454_end]
predictions_testset = np.genfromtxt(test_file, delimiter=',')       
coords_start = predictions_testset[:,5:7]
coords_end = predictions_testset[:,7:9]
time_start = time_input_testset[:]
passengers = predictions_testset[:,4]
taz_start = predictions_testset[:,9]
taz_end = predictions_testset[:,10]

In [ ]:
predicted_fares_testset = []

for row in range(len(predictions_testset)):
    predicted_fares_testset.append(TFP.TaxiFare((coords_start[row,0],coords_start[row,1]), (coords_end[row,0],coords_end[row,1]),
                   time_start[row], passengers[row], taz_start[row], taz_end[row],day[row])) 
#     if row%100==0:
#         print row

In [18]:
for row in range(10):
    print '$',format(predicted_fares_testset[row], '.2f')

$ 52.90
$ 7.70
$ 15.09
$ 8.40
$ 48.02
$ 8.74
$ 10.26
$ 12.12
$ 11.17
$ 10.43


### writing predictions to csv file

In [19]:
a = np.zeros(shape=[(len(predictions_testset)),2])
a[:,0] = predictions_testset[:,0] 
a[:,1] = predicted_fares_testset[:]

np.savetxt("predictions2.csv", a, fmt='%.2f', delimiter=",", 
           header= 'id,fare_predicted')

## Driver Advisory System

In [21]:
test_file = 'Taxi_Train.csv'
time_input_testset = []
to = []
with open(test_file) as fn: 
    for line in fn: # go over data file, line by line
        x = line.split(',') # get a list of attributes, as strings
        # parse the time strings from %m/%d/%Y %H:%M format into seconds
        tto = datetime.strptime(x[1], "%m/%d/%Y %H:%M")
        to.append((tto - tto.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds())

#  0-4    ['ID', 'STARTDATE', 'ENDDATE', 'Total_amount', 'PASSENGER_NUM',
#  5-10   'GPS_START_LO', 'GPS_START_LA', 'GPS_END_LO', 'GPS_END_LA', 'StartTAZ1454', 'TAZ1454_end]
predictions_testset = np.genfromtxt(test_file, delimiter=',')       
coords_start = predictions_testset[:,5:7]
coords_end = predictions_testset[:,7:9]
time_start = to[:]
passengers = predictions_testset[:,4]
taz_start = predictions_testset[:,9]
taz_end = predictions_testset[:,10]

In [27]:
u = set(taz_start)
y = set(taz_end)
u = list(u)
y = list(y)
print len(u) , 'distinct TAZs at start'
print len(y) , 'distinct TAZs at end'

226 distinct TAZs at start
581 distinct TAZs at end


In [28]:
taxi_counts_o = np.zeros(shape=[(len(u)),2])
for i in range(len(u)):
    taxi_counts_o[i,0] = int(u[i])
    for j in range(len(taz_start)):
        if taz_start[j]==taxi_counts_o[i,0]:
            taxi_counts_o[i,1] += 1   

z = taxi_counts_o[taxi_counts_o[:,1].argsort()]
print 'TAZ =', z[len(u)-1,0] , 'was the most popular origin with' , z[len(u)-1,1] , 'rides.'

taxi_counts_d = np.zeros(shape=[(len(y)),2])
for i in range(len(y)):
    taxi_counts_d[i,0] = int(y[i])
    for j in range(len(taz_end)):
        if taz_end[j]==taxi_counts_d[i,0]:
            taxi_counts_d[i,1] += 1   

z = taxi_counts_d[taxi_counts_d[:,1].argsort()]
print 'TAZ =', z[len(y)-1,0] , 'was the most popular destination with' , z[len(y)-1,1] , 'rides.'

    

TAZ = 239.0 was the most popular origin with 3696.0 rides.
TAZ = 239.0 was the most popular destination with 2813.0 rides.
